In [ ]:
import telebot
from telebot import TeleBot
from telebot import types
from time import sleep
import os
import codecs
import logging
import sqlite3
from datetime import datetime, timedelta
from dbworker import *
import numpy as np
import requests
import json
import time
from hashlib import sha256

logging.basicConfig(filename="botlog.log", level=logging.INFO, filemode='w')
token = '1618504886:AAFfCCV23hBgKqgDVScZkReEN7JntUevKdg'

logging.info('Started')

bot = TeleBot(token) 

mainMarkup = types.ReplyKeyboardMarkup(resize_keyboard=True)
bookCourt = types.KeyboardButton('Забронировать корт')
bookTraining = types.KeyboardButton('Записаться на тернировку')
bookGame = types.KeyboardButton('Записаться на игру')
myBookings = types.KeyboardButton('Мои брони')
mainMarkup.row(bookCourt)
mainMarkup.row(bookTraining)
mainMarkup.row(bookGame)
mainMarkup.row(myBookings)

backMarkup = types.ReplyKeyboardMarkup(row_width=1, resize_keyboard=True)
backBtn = types.KeyboardButton('Вернуться в главное меню')
backMarkup.add(backBtn)

approveMarkup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=1)
approveMarkup.row('Подтвердить')
approveMarkup.row('Вернуться в главное меню')
    
adminPanel = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2)
adminPanel.row('Просмотреть запись')
adminPanel.row('Создать тренировку')
adminPanel.row('Создать игру')

yesno = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
yesno.row('Да', 'Нет')
    
removeMarkup = types.ReplyKeyboardRemove(selective=False)

admin_list = [105565794, 431428976]

request = {}
all_users = mdb.users.find({})
for user in all_users:
    request[user['userId']] = {}

@bot.message_handler(func = lambda x: x.text in ['Вернуться в главное меню', 'Отмена'] and get_current_state(mdb, x.from_user.id) > 0)
def main_menu(message):
    global request
    request[message.chat.id] = {}
    user = search_or_save_user(mdb, message.from_user)
    set_state(mdb, user['userId'], 1)
    bot.send_message(message.chat.id, 'Главное меню', reply_markup=mainMarkup)
    return

@bot.message_handler(commands = ['start'])
def start_command(message):
    user = search_or_save_user(mdb, message.from_user)
    set_state(mdb, user['userId'], 1)
    greetings = 'Привет, это бот Global Sport Arena. С помощью него можно записаться на тренировки, бронировать корты и узнавать о последних новостях'
    rm = mainMarkup
    if user['state'] == 0:
        greetings += '\n\nЯ вижу, Вы у нас в первый раз, давайте пройдем коротенькую процедуру регистрации. Укажите ваше имя для записи на события'
        rm = removeMarkup
        bot.register_next_step_handler(message, register)
    bot.send_message(message.chat.id, greetings, reply_markup=rm)
    return

def register(message):
    global request
    request[message.chat.id] = {}
    update_user_field(mdb, message.from_user.id, 'nick', message.text)
    set_state(mdb, message.from_user.id, 1)
    bot.send_message(message.chat.id, 'Вы успешно зарегистрированы. Можете начинать пользоваться ботом!', reply_markup=mainMarkup)
    return

@bot.message_handler(func = lambda x: ('Забронировать' in x.text or 'Записаться' in x.text) and get_current_state(mdb, x.from_user.id) > 0)
def booking_init(message):
    if message.text == 'Вернуться в главное меню':
        main_menu(message)
        return
    global request
    request[message.chat.id] = {}
    msg = 'Пришлите желаемую дату в формате дд.мм.гг'
    if message.text == 'Забронировать корт':
        request[message.chat.id]['type'] = 0 
        msg += ' чч'
    elif message.text == 'Записаться на тернировку':
        request[message.chat.id]['type'] = 1
    elif message.text == 'Записаться на игру':
        request[message.chat.id]['type'] = 2
    else:
        bot.send_message(message.chat.id, 'Что то пошло не так, попробуйте позже', reply_markup=mainMarkup)
        return
    bot.send_message(message.chat.id, msg, reply_markup=backMarkup)
    bot.register_next_step_handler(message, get_date)
    return

@bot.message_handler(func = lambda x: ('Мои брони' in x.text) and get_current_state(mdb, x.from_user.id) > 0)
def my_bookings_init(message):
    if message.text == 'Вернуться в главное меню':
        main_menu(message)
        return
    global request
    request[message.chat.id] = {}
    show_bookings(message)
    return

def get_date(message):
    if message.text == 'Вернуться в главное меню':
        main_menu(message)
        return
    global request
    if request[message.chat.id]['type'] == 0:
        try:
            date=datetime.strptime(message.text, "%d.%m.%y %H")
            request[message.chat.id]['date'] = date
        except Exception as e:
            bot.send_message(message.chat.id, 'Неверный формат. Попробуйте в этом формате дд.мм.гг чч', reply_markup=backMarkup)
            bot.register_next_step_handler(message, get_date)
            return
        msg = 'Напишите продолжительность брони в часах, одним числом'
        mup = backMarkup
        next_step = get_booking_duration
    else:
        try:
            date=datetime.strptime(message.text, "%d.%m.%y")
            request[message.chat.id]['date'] = date
            date = date + timedelta(days = 1)
        except Exception as e:
            bot.send_message(message.chat.id, 'Неверный формат. Попробуйте в этом формате дд.мм.гг', reply_markup=backMarkup)
            bot.register_next_step_handler(message, get_date)
            return
        msg = 'Выберите Ваш уровень'
        mup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=1)
        mup.row('Начальный')
        mup.row('Средний')
        mup.row('Высокий')
        mup.row('Вернуться в главное меню')
        next_step = get_level
    if date < datetime.now():
        bot.send_message(message.chat.id, 'Время уже прошло, попробуйте еще раз', reply_markup=backMarkup)
        bot.register_next_step_handler(message, get_date)
        return
    bot.send_message(message.chat.id, msg, reply_markup=mup)
    bot.register_next_step_handler(message, next_step)    
    return
    
def get_booking_duration(message):
    global request
    if message.text == 'Вернуться в главное меню':
        main_menu(message)
        return
    try:
        duration = int(message.text)
    except:
        bot.send_message(message.chat.id, 'Что-то пошло не так, попробуйте еще раз')
        bot.register_next_step_handler(message, get_booking_duration)
        return
    request[message.chat.id]['duration'] = duration
    start = request[message.chat.id]['date']
    end = start + timedelta(hours=duration-1)
    bookings = mdb.bookings.find({})
    courts = set([c['courtId'] for c in mdb.courts.find({})])
    for booking in bookings:
        if booking['start'] >= start and booking['start'] <= end or booking['end'] >= start and booking['end'] <= end:
            try:
                courts.remove(booking['court'])
            except KeyError:
                continue
    if len(courts) == 0:
        bot.send_message(message.chat.id, 'Текущий диапазон занят. Попробуйте бронировать на меньшее время. Введите новое число часов', reply_markup=backMarkup)
        bot.register_next_step_handler(message, get_booking_duration)
        return
    request[message.chat.id]['court'] = list(courts)[0]
    request[message.chat.id]['price'] = str(get_booking_cost(start, end))
    msg = 'Проверьте составленную бронь. Если все в порядке, подтвердите, или забронируйте заново\n\n'
    msg += '<strong>Корт</strong>: ' + str(request[message.chat.id]['court'])
    msg += '\n<strong>Дата и время начала брони</strong>: ' + str(start)
    msg += '\n<strong>Дата и время конца брони</strong>: ' + str(end + timedelta(hours=1))
    msg += '\n<strong>Имя бронирующего</strong>: ' + search_or_save_user(mdb, message.from_user)['nick']
    msg += '\n<strong>Стоимость</strong>: ' + request[message.chat.id]['price']
    bot.send_message(message.chat.id, msg, reply_markup=approveMarkup, parse_mode='HTML')
    bot.register_next_step_handler(message, booking_approval)
    return

def booking_approval(message):
    global request
    booking = {}
    if message.text == 'Подтвердить':
        bookingId = np.random.randint(1, 10000)
        while check_booking(mdb, bookingId) is not None:
            bookingId = np.random.randint(1, 10000)
        booking['bookingId'] = bookingId
        booking['start'] = request[message.chat.id]['date']
        booking['duration'] = request[message.chat.id]['duration']
        booking['end'] = booking['start'] + timedelta(hours = request[message.chat.id]['duration'] - 1)
        booking['nick'] = search_or_save_user(mdb, message.from_user)['nick']
        booking['court'] = request[message.chat.id]['court']
        booking['price'] = request[message.chat.id]['price']
        booking['type'] = 'Бронь корта'
        booking['date'] = datetime(booking['start'].year, booking['start'].month, booking['start'].day)
        booking['hour'] = booking['start'].hour
        booking['user'] = message.from_user.id
        add_booking_to_court(mdb, booking['court'], bookingId)
        mdb.bookings.insert_one(booking)
        user = search_or_save_user(mdb, message.from_user)
        add_event(mdb, user['userId'], bookingId)
        bot.send_message(message.chat.id, 'Бронь сохранена. Назовите ваше имя адиминистратору на арене для оплаты брони', reply_markup=mainMarkup)
    else:
        main_menu(message)
    return
    
    
    
def get_booking_cost(start, end):
    cost = 0
    def locc(dt, h):
        if dt.weekday() < 5:
            if h >=7 and h < 15:
                cost = 1500
            elif h >= 15 and h < 17:
                cost = 1700
            elif h >= 17 and h < 19:
                cost = 2500
            elif h >=19 and h < 23:
                cost = 3000
            elif h < 7:
                cost = 2500
            elif h == 23: 
                cost = 2500
        else:
            cost = 2700
        return cost
    if start.hour <= end.hour:
        for h in range(start.hour, end.hour + 1):
            cost += locc(start, h)
    else:
        for h in range(0, end.hour + 1):
            cost += locc(end, h)
        for h in range(start.hour, 24):
            cost += locc(start, h)
    return cost

def get_level(message):
    global request
    if message.text == 'Вернуться в главное меню':
        main_menu(message)
        return
    if message.text in ['Начальный', 'Средний', 'Высокий']:
        request[message.chat.id]['level'] = message.text
    else:
        bot.send_message(message.chat.id, 'Что-то пошло не так, попробуйте еще раз')
        main_menu(message)
        return
    user = search_or_save_user(mdb, message.from_user)
    if request[message.chat.id]['type'] == 1:
        data = mdb.trainings.find({'date' : request[message.chat.id]['date'], 'level': request[message.chat.id]['level']})
        data = sorted(data, key = lambda x : x['hour'])
        data = list(filter(lambda x: x['free_places'] > 0, data))
        for x in data:
            msg = '<strong>' + x['header'] + '</strong>\n' if 'header' in x.keys() else ''
            inlines = types.InlineKeyboardMarkup(row_width=1)
            inlines.row(types.InlineKeyboardButton('Записаться', 
                                                   callback_data='training|' + str(x['trainId']) + '|on'))
            msg = ''
            bot.send_message(message.chat.id, msg + str(x['hour']) + ':00\n<strong>Длительность</strong>: ' + str(x['duration']) +
                            ' час(а)\n<strong>Тренер</strong>: ' + x['coach'] + '\n<strong>Корт</strong>: ' +
                            str(x['court']) + '\n<strong>Стоимость</strong>: ' + str(x['price']) + 
                             '\n<strong>Кол-во человек</strong>: ' + str(x['max_players']), reply_markup=inlines, parse_mode='HTML')
            
    else:
        data = mdb.games.find({'date' : request[message.chat.id]['date'], 'level': request[message.chat.id]['level']})
        data = sorted(data, key = lambda x : x['hour'])
        data = list(filter(lambda x: x['free_places'] > 0, data))
        for x in data:
            inlines = types.InlineKeyboardMarkup(row_width=1)
            inlines.row(types.InlineKeyboardButton('Записаться', 
                                                   callback_data='game|' + str(x['gameId']) + '|on'))
            msg = '<strong>' + x['header'] + '</strong>\n' if 'header' in x.keys() else ''
            bot.send_message(message.chat.id, msg + str(x['hour']) + ':00\n<strong>Длительность</strong>: ' + str(x['duration']) +
                            ' час(а)\n<strong>Корт</strong>: ' +
                            str(x['court']) + '\n<strong>Стоимость</strong>: ' + str(x['price']) + 
                             '\n<strong>Кол-во человек</strong>: ' + str(x['max_players']), reply_markup=inlines, parse_mode='HTML')
    set_state(mdb, user['userId'], 2)
    bot.send_message(message.chat.id, 'Выберите мероприятие', reply_markup=backMarkup)
    return

@bot.callback_query_handler(func = lambda call: call.data.split('|')[0] == 'training' and call.data.split('|')[2] == 'on' and get_current_state(mdb, call.from_user.id) == 2)
def onboard_training(call):
    global request
    request[call.from_user.id]['training']  = int(call.data.split('|')[1])
    training = mdb.trainings.find_one({'trainingId' : int(call.data.split('|')[1])})
    msg = 'Ваша бронь - тренировка:\n<strong>Дата</strong>: ' +  str(training['date']).split(' ')[0] +\
    ' ' + str(training['hour']) + ':00\n<strong>Длительность</strong>: ' + str(training['duration']) + \
    '\n<strong>Тренер</strong>: ' + training['coach'] + '\n<strong>Корт</strong>: ' + str(training['court']) + \
    '\n<strong>Стоимость</strong>: ' + str(training['price']) + '\n<strong>Текущие записанные</strong>:'
    for player in training['players']:
        temp = mdb.users.find_one({'userId': player})
        msg += '\n' + temp['nick']
    set_state(mdb, call.from_user.id, 1)
    bot.send_message(call.from_user.id, msg, reply_markup=approveMarkup, parse_mode='HTML')
    bot.register_next_step_handler(call.message, confirm_training)
    return

@bot.callback_query_handler(func = lambda call: call.data.split('|')[0] == 'game' and call.data.split('|')[2] == 'on' and get_current_state(mdb, call.from_user.id) == 2)
def onboard_game(call):
    global request
    request[call.from_user.id]['game']  = int(call.data.split('|')[1])
    game = mdb.games.find_one({'gameId' : int(call.data.split('|')[1])})
    msg = 'Ваша бронь - игра:\n<strong>Дата</strong>: ' +  str(game['date']).split(' ')[0] +\
    ' ' + str(game['hour']) + ':00\n<strong>Длительность</strong>: ' + str(game['duration']) + \
    '\n<strong>Корт</strong>: ' + str(game['court']) + '\n<strong>Уровень</strong>: ' + game['level'] + \
    '\n<strong>Стоимость</strong>: ' + str(game['price']) + '\n<strong>Текущие записанные</strong>:'
    for player in game['players']:
        temp = search_or_save_user(mdb, player)
        msg += '\n' + temp['nick']
    set_state(mdb, call.from_user.id, 1)
    bot.send_message(call.from_user.id, msg, reply_markup=approveMarkup, parse_mode='HTML')
    bot.register_next_step_handler(call.message, confirm_game)
    return
    
def confirm_training(message):
    global request
    if message.text == 'Подтвердить':
        user = search_or_save_user(mdb, message.from_user)
        training = get_training(mdb, request[message.chat.id]['training'])
        training['players'].append(message.from_user.id)
        update_training_field(mdb, training['trainId'], 'free_places', training['free_places'] - 1)
        update_training_field(mdb, training['trainId'], 'players', training['players'])
        add_event(mdb, message.from_user.id, training['bookingId'])
        bot.send_message(message.chat.id, 'Бронь сохранена. Назовите ваше имя адиминистратору на арене для оплаты брони', reply_markup=mainMarkup)
    else:
        main_menu(message)
    return

def confirm_game(message):
    global request
    if message.text == 'Подтвердить':
        user = search_or_save_user(mdb, message.from_user)
        game = get_game(mdb, request[message.chat.id]['game'])
        game['players'].append(message.from_user.id)
        update_game_field(mdb, game['gameId'], 'free_places', game['free_places'] - 1)
        update_game_field(mdb, game['gameId'], 'players', game['players'])
        add_event(mdb, message.from_user.id, game['bookingId'])
        bot.send_message(message.chat.id, 'Бронь сохранена. Назовите ваше имя адиминистратору на арене для оплаты брони', reply_markup=mainMarkup)
    else:
        main_menu(message)
    return

@bot.message_handler(commands = ['admin'])
def set_admin(message):
    if message.from_user.id not in admin_list:
        return
    set_state(mdb, message.from_user.id, -1)
    request[message.chat.id] = {}
    bot.send_message(message.chat.id, 'Панель админа', reply_markup=adminPanel)
    return

@bot.message_handler(commands = ['user'])
def set_user(message):
    if message.from_user.id not in admin_list:
        return
    set_state(mdb, message.from_user.id, 1)
    request[message.chat.id] = {}
    bot.send_message(message.chat.id, 'Панель пользователя', reply_markup=mainMarkup)
    return
    
@bot.message_handler(func = lambda x : get_current_state(mdb, x.from_user.id) < 0)
def admin_select(message):
    global request
    msg = 'Выберите дату в формате дд.мм.гг'
    if message.text == 'Просмотреть запись':
        request[message.chat.id]['type'] = 0
    elif message.text == 'Создать тренировку':
        request[message.chat.id]['type'] = 1
        msg += ' чч'
    elif message.text == 'Создать игру':
        request[message.chat.id]['type'] = 2
        msg += ' чч'
    elif message.text == 'Вернуться в главное меню':
        bot.send_message(message.chat.id, 'Главное меню', reply_markup=adminPanel)
        return
    bot.send_message(message.chat.id, msg, reply_markup=backMarkup)
    bot.register_next_step_handler(message, admin_get_date)
    return

def admin_get_date(message):
    if message.text == 'Вернуться в главное меню':
        bot.send_message(message.chat.id, 'Главное меню', reply_markup=adminPanel)
        return
    global request
    if request[message.chat.id]['type'] == 0:
        try:
            date=datetime.strptime(message.text, "%d.%m.%y")
        except Exception as e:
            bot.send_message(message.chat.id, 'Неверный формат. Попробуйте в этом формате дд.мм.гг', reply_markup=backMarkup)
            bot.register_next_step_handler(message, admin_get_date)
            return
        bookings = mdb.bookings.find({'date': date})
        bookings = sorted(bookings, key = lambda x: x['hour'])
        for booking in bookings:
            msg = '<strong>Тип</strong>: ' + booking['type']
            if booking['type'] == 'Бронь корта':
                msg += '\n<strong>Начало</strong>: ' + str(booking['start']) + '\n<strong>Конец</strong>: ' + \
                str(booking['end'] + timedelta(hours=1)) + '\n<strong>Бронирующий</strong>: ' + booking['nick'] + \
                '\n<strong>Стоимость</strong>: ' + booking['price']
                idd = booking['bookingId']
            else:
                msg += '\n<strong>Начало</strong>: ' + str(booking['start']) + '\n<strong>Конец</strong>: ' + \
                str(booking['end'] + timedelta(hours=1)) + '\n<strong>Корт</strong>: ' + str(booking['court']) + \
                '\n<strong>Уровень</strong>: ' + booking['level'] + '\n<strong>Записанные</strong>: '
                if booking['type'] == 'Тренировка':
                    training = get_training(mdb, booking['trainId'])
                    max_players = str(training['max_players'])
                    players = training['players']
                    idd = booking['trainId']
                    for player in players:
                        user = mdb.users.find_one({'userId': player})
                        msg += '\n' + user['nick'] 
                    msg += '\n<strong>Макс. кол-во</strong>: ' + str(max_players)
                    msg += '\n<strong>Тренер</strong>: ' + training['coach']
                elif booking['type'] == 'Игра':
                    game = get_game(mdb, booking['gameId'])
                    max_players = str(game['max_players'])
                    players = game['players']
                    idd = booking['gameId']
                    for player in players:
                        user = mdb.users.find_one({'userId': player})
                        msg += '\n' + user['nick'] 
                    msg += '\n<strong>Макс. кол-во</strong>: ' + str(max_players)
            deleteBtn = types.InlineKeyboardMarkup(row_width=1)
            deleteBtn.row(types.InlineKeyboardButton('Удалить', callback_data='delete|' + booking['type'] + '|' + str(idd)))
            bot.send_message(message.chat.id, msg, reply_markup=deleteBtn, parse_mode='HTML')
    else:
        try:
            date=datetime.strptime(message.text, "%d.%m.%y %H")
        except Exception as e:
            bot.send_message(message.chat.id, 'Неверный формат. Попробуйте в этом формате дд.мм.гг чч', reply_markup=backMarkup)
            bot.register_next_step_handler(message, admin_get_date)
            return
        if date < datetime.now():
            bot.send_message(message.chat.id, 'Дата прошла, попробуйте еще раз', reply_markup=backMarkup)
            bot.register_next_step_handler(message, admin_get_date)
            return
        bot.send_message(message.chat.id, 'Введите кол-во часов', reply_markup=backMarkup)
        bot.register_next_step_handler(message, admin_get_duration)
    request[message.chat.id]['date'] = date
    return
    
def admin_get_duration(message):
    global request
    if message.text == 'Вернуться в главное меню':
        set_admin(message)
        return
    try:
        duration = int(message.text)
    except:
        bot.send_message(message.chat.id, 'Что-то пошло не так, попробуйте еще раз')
        bot.register_next_step_handler(message, admin_get_duration)
        return
    request[message.chat.id]['duration'] = duration
    start = request[message.chat.id]['date']
    end = start + timedelta(hours=duration-1)
    bookings = mdb.bookings.find({})
    courts = set([c['courtId'] for c in mdb.courts.find({})])
    for booking in bookings:
        if (booking['start'] >= start and booking['start'] <= end) or (booking['end'] >= start and booking['end'] <= end):
            try:
                courts.remove(booking['court'])
            except KeyError:
                continue
    if len(courts) == 0:
        bot.send_message(message.chat.id, 'Текущий диапазон занят. Попробуйте указать другое время', reply_markup=backMarkup)
        bot.register_next_step_handler(message, admin_get_date)
        return
    request[message.chat.id]['court'] = list(courts)[0]
    mup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=1)
    mup.row('Начальный')
    mup.row('Средний')
    mup.row('Высокий')
    mup.row('Вернуться в главное меню')
    bot.send_message(message.chat.id, 'Подобран корт номер ' + str(list(courts)[0]) + '. Выберите уровень участников события', reply_markup=mup)
    bot.register_next_step_handler(message, admin_get_level)
    return

def admin_get_level(message):
    global request
    if message.text == 'Вернуться в главное меню':
        set_admin(message)
        return
    if message.text in ['Начальный', 'Средний', 'Высокий']:
        request[message.chat.id]['level'] = message.text
    else:
        bot.send_message(message.chat.id, 'Что-то пошло не так, попробуйте еще раз')
        set_admin(message)
        return
    bot.send_message(message.chat.id, 'Введите максимальное число игроков', reply_markup=backMarkup)
    bot.register_next_step_handler(message, admin_get_max_players)
    return

def admin_get_max_players(message):
    global request
    if message.text == 'Вернуться в главное меню':
        set_admin(message)
        return
    try:
        request[message.chat.id]['max_players'] = int(message.text)
    except:
        bot.send_message(message.chat.id, 'Что-то пошло не так, попробуйте еще раз')
        bot.register_next_step_handler(message, admin_get_max_players)
        return
    bot.send_message(message.chat.id, 'Введите стоимость за событие', reply_markup=backMarkup)
    bot.register_next_step_handler(message, admin_get_price)
    return

def admin_get_price(message):
    global request
    if message.text == 'Вернуться в главное меню':
        set_admin(message)
        return
    try:
        request[message.chat.id]['price'] = int(message.text)
    except:
        bot.send_message(message.chat.id, 'Что-то пошло не так, попробуйте еще раз')
        bot.register_next_step_handler(message, admin_get_price)
        return
    if request[message.chat.id]['type'] == 2:
        bot.send_message(message.chat.id, 'Добавить название к событию?', reply_markup=yesno)
        bot.register_next_step_handler(message, admin_add_header_choice)
    elif request[message.chat.id]['type'] == 1:
        bot.send_message(message.chat.id, 'Введите ФИО тренера', reply_markup=backMarkup)
        bot.register_next_step_handler(message, admin_get_coach)
    return

def admin_get_coach(message):
    global request
    if message.text == 'Вернуться в главное меню':
        set_admin(message)
        return
    request[message.chat.id]['coach'] = message.text
    bot.send_message(message.chat.id, 'Добавить название к событию?', reply_markup=yesno)
    bot.register_next_step_handler(message, admin_add_header_choice)
    return

def admin_add_header_choice(message):
    global request
    if message.text == 'Да':
        bot.send_message(message.chat.id, 'Введите название', reply_markup=backMarkup)
        bot.register_next_step_handler(message, admin_get_header)
    elif message.text == 'Нет':
        admin_before_approval(message)
    return
        
def admin_before_approval(message):
    global request
    msg = 'Событие: \n<strong>Дата</strong>: ' +  str(request[message.chat.id]['date']) +\
    '\n<strong>Длительность</strong>: ' + str(request[message.chat.id]['duration']) + \
    '\n<strong>Корт</strong>: ' + str(request[message.chat.id]['court']) + '\n<strong>Уровень</strong>: ' + request[message.chat.id]['level'] + \
    '\n<strong>Стоимость</strong>: ' + str(request[message.chat.id]['price']) + '\n<strong>Макс кол-во человек</strong>: ' + \
    str(request[message.chat.id]['max_players'])
    if 'header' in request[message.chat.id]:
        msg += '\n<strong>Название</strong>: ' + request[message.chat.id]['header']
    if request[message.chat.id]['type'] == 1:
        msg += '\n<strong>Тренер</strong>: ' + request[message.chat.id]['coach']
    bot.send_message(message.chat.id, msg, reply_markup=approveMarkup, parse_mode='HTML')
    bot.register_next_step_handler(message, admin_approval_itself)
    return

def admin_get_header(message):
    global request
    if message.text == 'Вернуться в главное меню':
        set_admin(message)
        return
    request[message.chat.id]['header'] = message.text
    admin_before_approval(message)
    return

def admin_approval_itself(message):
    global request
    booking = {}
    bookingId = np.random.randint(1, 10000)
    while check_booking(mdb, bookingId) is not None:
        bookingId = np.random.randint(1, 10000)
    booking['bookingId'] = bookingId
    if message.text == 'Подтвердить':
        booking['start'] = request[message.chat.id]['date']
        booking['duration'] = request[message.chat.id]['duration']
        booking['end'] = booking['start'] + timedelta(hours = request[message.chat.id]['duration'] - 1)
        booking['court'] = request[message.chat.id]['court']
        booking['price'] = request[message.chat.id]['price']
        booking['date'] = datetime(booking['start'].year, booking['start'].month, booking['start'].day)
        booking['hour'] = booking['start'].hour

        if 'header' in request[message.chat.id].keys():
            booking['header'] = request[message.chat.id]['header']
        booking['level'] = request[message.chat.id]['level']
        booking['max_players'] = request[message.chat.id]['max_players']
        booking['players'] = []
        booking['free_places'] = booking['max_players']
        
        if request[message.chat.id]['type'] == 1:
            booking['type'] = 'Тренировка'
            booking['coach'] = request[message.chat.id]['coach']
            trainId = np.random.randint(1, 10000)
            while get_training(mdb, trainId) is not None:
                trainId = np.random.randint(1, 10000)
            booking['trainId'] = trainId
            mdb.trainings.insert_one(booking)
        elif request[message.chat.id]['type'] == 2:
            booking['type'] = 'Игра'
            gameId = np.random.randint(1, 10000)
            while get_game(mdb, gameId) is not None:
                gameId = np.random.randint(1, 10000)
            booking['gameId'] = gameId
            mdb.games.insert_one(booking)
        mdb.bookings.insert_one(booking)
        bot.send_message(message.chat.id, 'Событие сохранено', reply_markup=adminPanel)
    else:
        set_admin(message)
    return

@bot.callback_query_handler(func = lambda call: call.data.split('|')[0] == 'delete' and get_current_state(mdb, call.from_user.id) < 0)
def admin_delete_event(call):
    booking = get_booking(mdb, int(call.data.split('|')[2]))
    if call.data.split('|')[1] == 'Бронь корта':
        bot.send_message(booking['user'], booking['type'] + ' дата: ' + str(booking['date']) + ' отменена по техническим причинам')
        mdb.bookings.delete_one({'bookingId': booking['bookingId']})
    else:
        if call.data.split('|')[1] == 'Тренировка':
            training = get_training(mdb, booking['trainId'])
            for player in training['players']:
                bot.send_message(player, 'Событие ' + booking['type'] + ' дата: ' + str(booking['date']) + ' отменено')
            mdb.trainings.delete_one({'trainId': booking['trainId']})
            mdb.bookings.delete_one({'bookingId': booking['bookingId']})
        elif call.data.split('|')[1] == 'Игра':
            game = get_game(mdb, booking['gameId'])
            for player in game['players']:
                bot.send_message(player, 'Событие ' + booking['type'] + ' дата: ' + str(booking['date']) + ' отменено')
            mdb.games.delete_one({'gameId': booking['gameId']})
            mdb.games.delete_one({'gameId': booking['gameId']})
    return

def show_bookings(message):
    user = search_or_save_user(mdb, message.from_user)
    for event in user['events']:
        booking = get_booking(mdb, event)
        if booking['start'] >= datetime.now():
            rejectMarkup = types.InlineKeyboardMarkup(row_width=1)
            rejectMarkup.row(types.InlineKeyboardButton('Отменить', callback_data='reject|' + str(booking['bookingId'])))
            bot.send_message(message.chat.id, form_message_booking(booking['bookingId']), reply_markup=rejectMarkup, parse_mode='HTML')
    return

@bot.callback_query_handler(func = lambda call: call.data.split('|')[0] == 'reject' and get_current_state(mdb, call.from_user.id))
def reject_booking(call):
    global request
    request[call.from_user.id]['reject'] = int(call.data.split('|')[1])
    bot.send_message(call.from_user.id, 'Вы уверены, что хотите отменить событие?', reply_markup=yesno)
    bot.register_next_step_handler(call.message, approve_reject)
    
    
def approve_reject(message):
    global request
    if message.text == 'Да':
        booking = get_booking(mdb, request[message.from_user.id]['reject'])
        user = search_or_save_user(mdb, message.from_user)
        if 'trainId' in booking.keys():
            training = get_training(mdb, booking['trainId'])
            update_training_field(mdb, booking['trainId'], 'free_places', training['free_places'] + 1)
            temp = []
            for player in training['players']:
                if player != message.from_user.id:
                    temp.append(player)
            update_training_field(mdb, booking['trainId'], 'players', temp)
        elif 'gameId' in booking.keys():
            game = get_game(mdb, booking['gameId'])
            update_game_field(mdb, booking['gameId'], 'free_places', game['free_places'] + 1)
            temp = []
            for player in game['players']:
                if player != message.from_user.id:
                    temp.append(player)
            update_game_field(mdb, booking['gameId'], 'players', temp)
        else:
            mdb.booking.delete_one({'bookingId': request[message.from_user.id]['reject']})
        temp = []
        for event in user['events']:
            if event != booking['bookingId']:
                temp.append(event)
        update_user_field(mdb, user['userId'], 'events', temp)
        bot.send_message(message.chat.id, 'Событие успешно отменено', reply_markup=mainMarkup)
    else:
        main_menu(message)
    return

def form_message_booking(bookingId):
    booking = get_booking(mdb, bookingId)
    msg = '<strong>' + booking['header'] + '</strong>\n' if 'header' in booking.keys() else ''
    msg += '<strong>' + booking['type'] + '</strong>\n<strong>Дата</strong>: ' +  str(booking['date']).split(' ')[0] +\
    ' ' + str(booking['hour']) + ':00\n<strong>Длительность</strong>: ' + str(booking['duration']) + \
    '\n<strong>Корт</strong>: ' + str(booking['court']) + \
    '\n<strong>Стоимость</strong>: ' + str(booking['price']) 
    if booking['type'] != 'Бронь корта':
        msg += '\n<strong>Текущие записанные</strong>:'
        if 'gameId' in booking.keys():
            event = get_game(mdb, booking['gameId'])
            for player in event['players']:
                msg += mdb.users.find_one({'userId': player})['nick'] + '\n'
        elif 'trainId' in booking.keys():
            event = get_training(mdb, booking['trainId'])
            for player in event['players']:
                msg += mdb.users.find_one({'userId': player})['nick'] + '\n'
            msg += '<strong>Тренер</strong>: ' + booking['coach']
    return msg
                
bot.polling(none_stop=True)    